In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, Embedding, Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
#Inicializar y abrir archivo
corpus=[]
target=[]
edad=[]
sexo=[]
severidad=[]
archivo=open("dataset_elpino.csv",encoding="utf-8")

#Sacar las variables (features o característica)
header=archivo.readline().strip().split(";")
features=[]
for col in header:
    if col.startswith("Diag") or col.startswith("Proc"):
        col=col.split(" ")
        col=col[0]+col[1]
    features.append(col)

#Sacar la data (deja sólo los códigos en el corpus)
for linea in archivo:
    row=[]
    linea=linea.strip().split(";")
    for i in range(len(linea)):
        col=linea[i].split("-")[0].strip()
        if i==67:
            grd=col
            #target.append(grd[-1])
            target.append(grd)
        elif i==66:
            sexo.append(1 if col=="Mujer" else 0)
        elif i==65:
            edad.append(int(col))
        else:
            row.append(col)
    corpus.append(row)

In [4]:
# Crear el dataframe
df=pd.DataFrame(corpus,columns=features[:-3])
df["GRD"]=target
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

In [21]:
def construirModelo(n):
    filtro_lst = list(dict(df["GRD"].value_counts()[:]).keys())[:n]
    filtro={filtro_lst[j]: [1 if (filtro_lst[j] == i) else 0 for i in filtro_lst] for j in range(len(filtro_lst))}

    # Crear los X's y los Y's
    seed=12122008
    X=[]
    Y=[]
    vocabulario={}
    for linea in range(len(corpus)):
        message=[]
        if target[linea] in filtro.keys():
            for j in corpus[linea]:
                message.append(j)
                if j in vocabulario:
                    vocabulario[j]+=1
                else:
                    vocabulario[j]=1
            X.append(message)
            Y.append(filtro[target[linea]])

    # Tokenizar el texto
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    words=list(tokenizer.word_index.keys())

    vocabulary=len(words)+1
    X = sequences
    # Convertir los 1 en 0 para que no se cuenten como palabras
    sequences_array = np.array(X)
    sequences_array[sequences_array == 1] = 0
    X = sequences_array.tolist()

    input_length=len(X[0])

    X=np.asarray(X)
    Y=np.asarray(Y)
    x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=seed)
    modelo_gru = Sequential()
    modelo_gru.add(Embedding(input_dim=vocabulary, output_dim=64))  # Embedding para códigos
    modelo_gru.add(GRU(64, return_sequences=False, activation="relu"))  # GRU con 64 unidades
    modelo_gru.add(Dense(len(filtro_lst), activation="softmax"))  # Capa de salida
    modelo_gru.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return modelo_gru, x_train, y_train, x_val, y_val

In [22]:
y_pred_probs = dict()
for i in  range(20, 421, 20):
    print("Entrenando modelo con {} GRD".format(i))
    modelo, x_train, y_train, x_val, y_val = construirModelo(i)
    modelo.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32, verbose=0)
    print("Accuracy: ", modelo.history.history["val_accuracy"][-1])
    y_pred_probs[i] = modelo.predict(x_val)

Entrenando modelo con 20 GRD
Accuracy:  0.9428815245628357
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Entrenando modelo con 40 GRD
Accuracy:  0.9182782173156738
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Entrenando modelo con 60 GRD
Accuracy:  0.8622915744781494
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Entrenando modelo con 80 GRD
Accuracy:  0.8528265357017517
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Entrenando modelo con 100 GRD
Accuracy:  0.8526886701583862
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Entrenando modelo con 120 GRD
Accuracy:  0.849743127822876
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Entrenando modelo con 140 GRD
Accuracy:  0.7976141571998596
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Entrenando modelo con 160 GRD
Accuracy:  0.7831805348396301
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Entrenando modelo con 180 GRD
Accuracy:  0.747863233089447
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Entrenando modelo con 200 GRD
Accuracy:  0.7632279992103577
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Entrenando mod